In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhijeetbasant","key":"37f465ad16fd85d578c6bfdf8919a490"}'}

In [2]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/kaggle/kaggle.json

# download the dataset
!kaggle competitions download -c tensorflow-speech-recognition-challenge

chmod: cannot access '/root/kaggle/kaggle.json': No such file or directory
403 - Forbidden - You must accept this competition's rules before you'll be able to download files.


## Using tensorflow dataset

In [3]:
!pip install pydub

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the Speech Commands dataset
dataset_name = "speech_commands"
dataset, info = tfds.load(dataset_name, with_info=True, as_supervised=True)

train_data, test_data = dataset['train'], dataset['test']
print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/85511 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/speech_commands/incomplete.GBK4L1_0.0.3/speech_commands-train.tfrecord*...…

Generating validation examples...:   0%|          | 0/10102 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/speech_commands/incomplete.GBK4L1_0.0.3/speech_commands-validation.tfrecor…

Generating test examples...:   0%|          | 0/4890 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/speech_commands/incomplete.GBK4L1_0.0.3/speech_commands-test.tfrecord*...:…

Dataset speech_commands downloaded and prepared to /root/tensorflow_datasets/speech_commands/0.0.3. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='speech_commands',
    full_name='speech_commands/0.0.3',
    description="""
    An audio dataset of spoken words designed to help train and evaluate keyword
    spotting systems. Its primary goal is to provide a way to build and test small
    models that detect when a single word is spoken, from a set of ten target words,
    with as few false positives as possible from background noise or unrelated
    speech. Note that in the train and validation set, the label "unknown" is much
    more prevalent than the labels of the target words or background noise. One
    difference from the release version is the handling of silent segments. While in
    the test set the silence segments are regular 1 second files, in the training
    they are provided as long segments under "background_noise" folder. Here we
    split the

In [5]:
import librosa
import numpy as np

def preprocess_audio(audio, sample_rate=16000):
    # Convert tensor to numpy array and normalize
    audio = tf.cast(audio, tf.float32) / 32768.0
    audio_np = audio.numpy()

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=audio_np, sr=sample_rate, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)

# Preprocess the dataset
def prepare_dataset(data):
    features = []
    labels = []
    for audio, label in tfds.as_numpy(data):
        mfcc_features = preprocess_audio(audio)
        features.append(mfcc_features)
        labels.append(label)
    return np.array(features), np.array(labels)

X_train, y_train = prepare_dataset(train_data)
X_test, y_test = prepare_dataset(test_data)

In [ ]:
print(prepare_dataset.features)

In [24]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Train the GMM
gmm = GaussianMixture(n_components=len(np.unique(y_train_encoded)), covariance_type='diag')
gmm.fit(X_train, y_train_encoded)


GaussianMixture(covariance_type='diag', n_components=12)

In [25]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on test data
y_pred_encoded = gmm.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.07893660531697341
Classification Report:
               precision    recall  f1-score   support

           0       0.10      0.10      0.10       406
           1       0.12      0.07      0.09       402
           2       0.06      0.15      0.08       412
           3       0.01      0.01      0.01       405
           4       0.13      0.23      0.17       402
           5       0.09      0.08      0.08       396
           6       0.09      0.09      0.09       396
           7       0.00      0.00      0.00       411
           8       0.20      0.15      0.17       425
           9       0.05      0.03      0.04       419
          10       0.00      0.00      0.00       408
          11       0.08      0.04      0.06       408

    accuracy                           0.08      4890
   macro avg       0.08      0.08      0.07      4890
weighted avg       0.08      0.08      0.07      4890



In [26]:
import tensorflow as tf
import numpy as np
import IPython.display as ipd
import librosa

In [28]:
# Get one example from the training dataset
for audio, label in dataset.take(1): # Access the 'train' split of the dataset
    example_audio = audio.numpy()
    example_label = label.numpy()  # True label for reference

# print("Label:", example_label)
# Play the audio
ipd.Audio(example_audio, rate=16000)

In [29]:
# Preprocess the audio to extract MFCC features
def preprocess_audio_for_model(audio, sample_rate=16000):
    audio_float = tf.cast(audio, tf.float32) / 32768.0
    mfcc = librosa.feature.mfcc(y=audio_float.numpy(), sr=sample_rate, n_mfcc=13)
    return np.mean(mfcc.T, axis=0)

# Extract features
input_features = preprocess_audio_for_model(example_audio)
input_features = input_features.reshape(1, -1)  # Reshape for prediction


In [30]:
# Predict using the trained GMM model
predicted_label_encoded = gmm.predict(input_features)
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

print("Predicted Label:", predicted_label[0])
print("True Label (for reference):", label_encoder.inverse_transform([example_label])[0])


Predicted Label: 2
True Label (for reference): 4


In [31]:
# Load dataset info to get label names
dataset, info = tfds.load("speech_commands", split="train", as_supervised=True, with_info=True)
label_names = info.features['label'].names

print("Label Names:", label_names)  # This prints the list of labels (e.g., 'yes', 'no', etc.)


Label Names: ['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes', '_silence_', '_unknown_']


In [32]:
# Map numeric labels to words
predicted_word_label = label_names[predicted_label_encoded[0]]
true_word_label = label_names[example_label]

print("Predicted Label (Word):", predicted_word_label)
print("True Label (Word):", true_word_label)


Predicted Label (Word): left
True Label (Word): off
